# 3. Generating Multiple Samples using MS1 Controller

In this notebook, we demonstrate how ViMMS can be used to generate multiple samples (sets of chemicals) that are biological and technical replicates. The MS1 controller is then used to produce mass spectral data in form of .mzML files for the multiple samples.

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from collections import defaultdict
import os
from pathlib import Path

In [4]:
import sys
sys.path.append('..')

In [5]:
from vimms.Chemicals import ChemicalCreator, MultiSampleCreator
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import SimpleMs1Controller
from vimms.Common import *

Load previously trained KDEs in `PeakSampler` and the list of extracted metabolites, created in **01. Download Data.ipynb**.

In [6]:
base_dir = os.path.abspath('example_data')
ps = load_obj(Path(base_dir, 'peak_sampler_mz_rt_int_19_beers_fullscan.p'))
hmdb = load_obj(Path(base_dir, 'hmdb_compounds.p'))

Set ViMMS logging level

In [7]:
set_log_level_warning()
# set_log_level_info()
# set_log_level_debug()

## Create Initial Chemical

Define an output folder containing our results

In [8]:
out_dir = Path(base_dir, 'results', 'MS1_multiple')

Here we generate multiple chemical objects that will be used across samples. The chemical objects are generated by sampling from metabolites in the HMDB database.

In [9]:
# the list of ROI sources created in the previous notebook '01. Download Data.ipynb'
ROI_Sources = [str(Path(base_dir,'DsDA', 'DsDA_Beer', 'beer_t10_simulator_files'))]

# minimum MS1 intensity of chemicals
min_ms1_intensity = 1.75E5

# m/z and RT range of chemicals
rt_range = [(400, 800)]
mz_range = [(100, 400)]

# the number of chemicals in the sample
n_chems = 1000

# maximum MS level (we do not generate fragmentation peaks when this value is 1)
ms_level = 1

# for this experiment, we restrict the sampled chromatograms to be within 20 - 40s in length
# so they are not too big and too small
roi_rt_range = [20, 40]

In [10]:
chems = ChemicalCreator(ps, ROI_Sources, hmdb)
dataset = chems.sample(mz_range, rt_range, min_ms1_intensity, n_chems, ms_level, roi_rt_range=roi_rt_range)
save_obj(dataset, Path(out_dir, 'BaseDataset', 'dataset.p'))

Created /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/BaseDataset
Saving <class 'list'> to /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/BaseDataset/dataset.p


In [11]:
for chem in dataset[0:10]:
    print(chem)

KnownChemical - 'C8H7NO5' rt=576.17 max_intensity=1712866.07
KnownChemical - 'C14H13Cl2NS' rt=411.32 max_intensity=1791477.92
KnownChemical - 'C14H28O2' rt=534.73 max_intensity=63127138.37
KnownChemical - 'C9H14N4O5' rt=541.75 max_intensity=16601427.75
KnownChemical - 'C13H15NO2' rt=424.91 max_intensity=1044593.45
KnownChemical - 'C9H11N' rt=691.89 max_intensity=372585.92
KnownChemical - 'C4H5N3O2' rt=423.63 max_intensity=2925134.86
KnownChemical - 'C9H14N4O5' rt=762.06 max_intensity=101686530.98
KnownChemical - 'C6H11N3' rt=418.06 max_intensity=704583.22
KnownChemical - 'C10H20O2' rt=514.49 max_intensity=48521086.29


## Create Multiple Samples

The next section allows us to define classes of biological replicates, each having multiple technical replicates. 

Below we create two biological classes ('class0', 'class1'), each having 10 technical replicates with some noise on the chemical's intensity.

In [12]:
n_samples = [10, 10] # number of files per class
classes = ["class%d" % i for i in range(len(n_samples))] # creates default list of classes
intensity_noise_sd = [1000] # noise on max intensity

In [13]:
classes

['class0', 'class1']

Add intensity changes between different classes

In [14]:
change_probabilities = [0 for i in range(len(n_samples))] # probability of intensity changes between different classes
change_differences_means = [0 for i in range(len(n_samples))] # mean of those intensity changes
change_differences_sds = [0 for i in range(len(n_samples))] # SD of those intensity changes

Add experimental variables (examples in comments)

In [15]:
experimental_classes = None # [["male","female"],["Positive","Negative","Unknown"]]
experimental_probabilitities = None # [[0.5,0.5],[0.33,0.33,0.34]]
experimental_sds = None # [[250],[250]]

Dropout chemicals in different classes

In [16]:
# drop-out chemicals by their probabilities
dropout_probability = 0.2
dropout_probabilities = [dropout_probability for i in range(len(n_samples))]
dropout_numbers = None # drop-out chemicals by an absolute number

# dropout_probabilities = None
# dropout_numbers = 2 # number of chemicals dropped out in each class

Generate multiple samples

In [17]:
save_location = os.path.join(out_dir, 'ChemicalFiles')

In [18]:
multiple_samples = MultiSampleCreator(dataset, n_samples, classes, intensity_noise_sd, 
                                      change_probabilities, change_differences_means, change_differences_sds, dropout_probabilities, dropout_numbers,
                                     experimental_classes, experimental_probabilitities, experimental_sds, save_location=save_location)

Created /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/ChemicalFiles
Saving <class 'list'> to /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/ChemicalFiles/sample_0.p
Saving <class 'list'> to /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/ChemicalFiles/sample_1.p
Saving <class 'list'> to /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/ChemicalFiles/sample_2.p
Saving <class 'list'> to /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/ChemicalFiles/sample_3.p
Saving <class 'list'> to /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/ChemicalFiles/sample_4.p
Saving <class 'list'> to /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/ChemicalFiles/sample_5.p
Saving <class 'list'> to /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/ChemicalFiles/sample_6.p
Saving <class 'list'> to /home/joewandy/git/vimms/examples/example_data/results/MS1_

In [19]:
total_samples = np.sum(multiple_samples.n_samples)
total_samples

20

We can also print the chemicals that are missing (removed by drop-out) in each class.

In [20]:
save_obj(multiple_samples.missing_chemicals, Path(out_dir, 'MissingChemicals', 'missing_chemicals.p'))
multiple_samples.missing_chemicals

Created /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/MissingChemicals
Saving <class 'list'> to /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/MissingChemicals/missing_chemicals.p


[[KnownChemical - 'C10H20O2' rt=514.49 max_intensity=48521086.29,
  KnownChemical - 'C18H26ClN3' rt=614.81 max_intensity=1129816.23,
  KnownChemical - 'C4H9NO2S' rt=720.74 max_intensity=6696060.68,
  KnownChemical - 'C13H21N3O' rt=524.82 max_intensity=912498.95,
  KnownChemical - 'C7H7NO2' rt=711.11 max_intensity=720694.43,
  KnownChemical - 'C5H9NO3' rt=647.25 max_intensity=31602449.37,
  KnownChemical - 'C20H26N4O' rt=492.27 max_intensity=530032.29,
  KnownChemical - 'C13H17NO' rt=591.13 max_intensity=199037.73,
  KnownChemical - 'C14H11Cl2NO4' rt=664.94 max_intensity=636618.20,
  KnownChemical - 'C18H20FN3O4' rt=533.89 max_intensity=783502.51,
  KnownChemical - 'C20H21N' rt=491.97 max_intensity=7589974.64,
  KnownChemical - 'C7H7NO2' rt=467.99 max_intensity=2099935.21,
  KnownChemical - 'C12H18N2O2' rt=566.96 max_intensity=530559.66,
  KnownChemical - 'C12H14ClNO' rt=456.57 max_intensity=502209.27,
  KnownChemical - 'C8H14O3' rt=773.18 max_intensity=2728116.46,
  KnownChemical - 'C7

## Run MS1 controller on the samples and generate .mzML files

We can now take the multiple samples created above and generate mass spectral data (.mzML files) using the MS1 controller in ViMMS.

In [ ]:
min_rt = rt_range[0][0]
max_rt = rt_range[0][1]
controllers = defaultdict(list)
controller_to_mzml = {}

mzml_dir = Path(out_dir, 'mzMLFiles')
num_classes = len(n_samples)
sample_idx = 0
for j in range(num_classes): # loop over classes
    num_samples = n_samples[j]
    for i in range(num_samples): # loop over samples for each class
        
        # load the sample
        fname = Path(save_location, 'sample_%d.p' % sample_idx) 
        sample = load_obj(fname)
        sample_idx += 1
        
        # define output .mzML filename
        out_file = 'number_%d_class_%d.mzML' % (i, j)
        out_path = Path(mzml_dir, out_file)
        print('Generating %s' % out_path)

        # run it through the MS1 controller        
        mass_spec = IndependentMassSpectrometer(POSITIVE, sample, density=ps.density_estimator)
        controller = SimpleMs1Controller(mass_spec)
        controller.run(min_rt, max_rt)
        controller.write_mzML('my_analysis', out_path)

        # save the resulting controller
        controllers[j].append(controller)
        controller_to_mzml[controller] = (j, out_file, )

  4%|▍         | 16.966699999999832/400 [00:00<00:02, 157.92it/s]

Generating /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/mzMLFiles/number_0_class_0.mzML


401.1769830000021it [00:03, 107.70it/s]                          


Created /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/mzMLFiles


  4%|▍         | 17.497000000000185/400 [00:00<00:02, 172.94it/s]

...

400.3849299999997it [00:03, 115.53it/s]                          
  5%|▍         | 18.352000000000203/400 [00:00<00:02, 162.63it/s]

Generating /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/mzMLFiles/number_7_class_0.mzML


400.7003100000003it [00:03, 105.15it/s]                          
  4%|▍         | 17.858999999999924/400 [00:00<00:02, 174.61it/s]

Generating /home/joewandy/git/vimms/examples/example_data/results/MS1_multiple/mzMLFiles/number_8_class_0.mzML


 45%|████▌     | 181.64089999999896/400 [00:01<00:01, 139.17it/s]

## Print out the missing peaks

The controller object contains all the information about the state of the mass spectrometry process over time. Below we demonstrate this by generating a report of peaks corresponding to a chemical that are present in one class but is missing from the other class. This can be useful in the benchmark evaluation of peak picking or alignment algorithms.

In [ ]:
def get_chem_to_peaks(controller):
    chem_to_peaks = defaultdict(list)
    frag_events = controller.mass_spec.fragmentation_events
    for frag_event in frag_events:
        chem = frag_event.chem
        peaks = frag_event.peaks
        chem_to_peaks[chem].extend(peaks)
    return chem_to_peaks

In [ ]:
for controller, (current_class, mzml_filename) in controller_to_mzml.items():
    controller_peaks = get_chem_to_peaks(controller)
    basename = os.path.basename(mzml_filename)
    front, back = os.path.splitext(mzml_filename)
    outfile = front + '.csv'

    missing_peaks = []            
    for other_class in range(num_classes):
        if current_class == other_class:
            continue

        # get the peaks that are present in current_class but missing in other_class
        missing_chems = multiple_samples.missing_chemicals[other_class]
        for chem in missing_chems:
            peaks = controller_peaks[chem]
            for peak in peaks:
                row = (chem.formula.formula_string, current_class, other_class, peak.mz, peak.rt, peak.intensity)
                missing_peaks.append(row)
    
    # convert to dataframe
    columns = ['formula', 'present_in', 'missing_in', 'mz', 'RT', 'intensity']
    missing_df = pd.DataFrame(missing_peaks, columns=columns)
    missing_df.to_csv(os.path.join(out_dir, 'MissingChemicals', os.path.basename(outfile)))

In [ ]:
missing_df.head(20)